# Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

Using TensorFlow backend.


# Reading Dataset

In [2]:
IMAGE_SIZE = [224, 224]

train_path = '../dataset/train'
valid_path = '../dataset/valid'

# VGG19 Model

In [3]:
vgg19 = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

We don't want to train all layers so keeping that parameter equal to "False" except the last layer.

In [4]:
for layer in vgg19.layers:
    layer.trainable = False

In [5]:
folders = glob('../dataset/train/*')

In [6]:
folders

['../dataset/train/Tomato___Target_Spot',
 '../dataset/train/Tomato___Late_blight',
 '../dataset/train/Tomato___Tomato_mosaic_virus',
 '../dataset/train/Tomato___Leaf_Mold',
 '../dataset/train/Tomato___Bacterial_spot',
 '../dataset/train/Tomato___Early_blight',
 '../dataset/train/Tomato___healthy',
 '../dataset/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '../dataset/train/Tomato___Spider_mites Two-spotted_spider_mite',
 '../dataset/train/Tomato___Septoria_leaf_spot']

In [7]:
# our layers - you can add more if you want
x = Flatten()(vgg19.output)

In [8]:
len(folders)

10

Adding last layer with 10 outputs and using softmax as an activation. 

In [9]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg19.input, outputs=prediction)

# Model Summary

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

Using RMSProp as an optimizer with learning rate as 0.001. Keeping learning rate constant throughout.

In [11]:
# optimizer = RMSprop(0.001) #learning rate = 0.001

model.compile(
  loss = 'categorical_crossentropy',
  optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001),
  metrics = ['accuracy']
)

Create data generator for training and testing set. Augmentation process is only done for training dataset.

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('../dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('../dataset/valid',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 18345 images belonging to 10 classes.
Found 4585 images belonging to 10 classes.


In [17]:
# fit the model
#es = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

r = model.fit(
    training_set,
  validation_data=test_set,
  epochs=30,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)
)

Epoch 1/30
128/574 [=====>........................] - ETA: 1:51:59 - loss: 4.5845 - accuracy: 0.3087

KeyboardInterrupt: 

# Validation Accuracy

In [ ]:
valid_loss, valid_acc = model.evaluate_generator(test_set, steps=len(test_set))
print(f"Final validation accuracy: {valid_acc*100:.2f}%")

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('Acc-Val_acc')

In [ ]:
model.save('model_vgg19.h5')
print("Model saved successfully...")

# Prediction

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

# The End